In [6]:
#!/usr/bin/env python2
#
# Example to classify faces.
# Brandon Amos
# 2015/10/11

import time

start = time.time()

import argparse
#import cv2
import os
import pickle
import sys

from operator import itemgetter

import numpy as np
np.set_printoptions(precision=2)
import pandas as pd

#import openface

from sklearn.pipeline import Pipeline
from sklearn.lda import LDA
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.mixture import GMM
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split,  KFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
#fileDir = os.path.dirname(os.path.realpath(__file__))
#modelDir = os.path.join(fileDir, '..', 'models')
#dlibModelDir = os.path.join(modelDir, 'dlib')
#openfaceModelDir = os.path.join(modelDir, 'openface')


def train(classfier, data, labelsNum, nClasses,):
    labels = data[:,0]
    embeddings = data[:,1:]
    labelsNum = labels.tolist()
    print("Training for {} classes.".format(nClasses))
    if classifier == 'LinearSvm':
        clf = SVC(C=1, kernel='linear', probability=True)
    elif classifier == 'GridSearchSvm':
#         print("""
#         Warning: In our experiences, using a grid search over SVM hyper-parameters only
#         gives marginally better performance than a linear SVM with C=1 and
#         is not worth the extra computations of performing a grid search.
#         """)
        param_grid = [
            {'C': [1, 10, 100, 1000],
             'kernel': ['linear']},
            {'C': [1, 10, 100, 1000],
             'gamma': [0.001, 0.0001],
             'kernel': ['rbf']}
        ]
        clf = GridSearchCV(SVC(C=1, probability=True), param_grid, cv=5)
    # ref:
    # http://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#example-classification-plot-classifier-comparison-py
    elif classifier == 'DecisionTree':  # Doesn't work best
        clf = DecisionTreeClassifier(max_depth=20)
    elif classifier == 'KNN':
        clf = KNeighborsClassifier(n_neighbors=1)
    elif classifier == 'AdaBoost':
        clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=20), n_estimators=100)
    elif classifier == 'RandomForest':
        clf = RandomForestClassifier(n_estimators=10)
    
    start = time.time()
    clf.fit(embeddings, labelsNum)
    
    """
    ### saving model to local file
    fName = "{}/classifier.pkl".format(workDir)
    print("Saving classifier to '{}'".format(fName))
    with open(fName, 'w') as f:
        pickle.dump((le, clf), f)
    """
    
    return clf, (time.time() - start)

"""
"""
def infer(clf, X, Y, multiple=False, verbose=True):
    """
    classifierModel = "{}/classifier.pkl".format(workDir)
    with open(classifierModel, 'rb') as f:
        if sys.version_info[0] < 3:
                (le, clf) = pickle.load(f)
        else:
                (le, clf) = pickle.load(f, encoding='latin1')
    """

    # TODO Store testing represenations in folder 
    start = time.time()
    f_x = clf.predict(X)
    error = np.sum(Y[:,0] != f_x) / float(len(Y))

    print "\tTesting error is {}".format(error)
    return error,( time.time()-start)
    # reps = getRep(img, multiple)
    # if len(reps) > 1:
    #     print("List of faces in image from left to right")
    # for r in reps:
    #     rep = r[1].reshape(1, -1)
    #     bbx = r[0]

    #     start = time.time()
    #     predictions = clf.predict_proba(rep).ravel()
    #     maxI = np.argmax(predictions)
    #     person = le.inverse_transform(maxI)
    #     confidence = predictions[maxI]

    #     if verbose:
    #         print("Prediction took {} seconds.".format(time.time() - start))
    #     if multiple:
    #         print("Predict {} @ x={} with {:.2f} confidence.".format(person.decode('utf-8'), bbx,
    #                                                                  confidence))
    #     else:
    #         print("Predict {} with {:.2f} confidence.".format(person.decode('utf-8'), confidence))

    #     # match prediction with label
    #     # Sum up 

In [7]:
workDir = "./training-embeddings"
print("Loading embeddings.")
fname = "{}/labels.csv".format(workDir)
labels = pd.read_csv(fname, header=None).as_matrix()[:, 0:1]
fname = "{}/reps.csv".format(workDir)
embeddings = pd.read_csv(fname, header=None).as_matrix()
le = LabelEncoder().fit(labels)
labelsNum = le.transform(labels)
nClasses = len(le.classes_)

print embeddings.shape
print labels.shape, embeddings.shape
data = np.append(labels,embeddings,axis=1)
Y = data[:,0:1]
X = data[:,1:]

# Split dataset
# Train on generated embeddings
splits = [.20,.40,.50,.60,.80]  # percentage of test set
clf_list = ['GridSearchSvm','LinearSvm', 'DecisionTree', 'AdaBoost', 'KNN', 'RandomForest']

for split in splits:
    for clf_name in clf_list:
        print "----------------------------------------------------"
        print "Using classifier {}".format(clf_name)
        print "[{},{}] [Train,Test] Split".format(int(100-(split*100)),\
                                                int((split*100)))
        X_train, X_test, Y_train, Y_test = train_test_split(X,Y,
                                      test_size=split,random_state=42)
        classifier = clf_name
        data = np.append(Y_train, X_train, axis=1)
        clf, train_time = train(classifier, data, labelsNum, nClasses)
        
        if clf_name is clf_list[0]:
            grid_scores = clf.grid_scores_
            for (param, avg_validation_score, cv_scores) in grid_scores:
                print param
                print cv_scores
        print "Took {} seconds".format(train_time)
        test_error, infer_time = infer(clf, X_test, Y_test)
        print "Took {} seconds".format(infer_time)
        
        print "----------------------------------------------------"
        print



Loading embeddings.
(997, 128)
(997, 1) (997, 128)
----------------------------------------------------
Using classifier GridSearchSvm
[80,20] [Train,Test] Split
Training for 50 classes.
Took 53.8950090408 seconds
	Testing error is 0.015
Took 0.0481081008911 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier LinearSvm
[80,20] [Train,Test] Split
Training for 50 classes.
Took 0.760406017303 seconds
	Testing error is 0.015
Took 0.0481350421906 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier DecisionTree
[80,20] [Train,Test] Split
Training for 50 classes.
Took 0.28494977951 seconds
	Testing error is 0.455
Took 0.00279903411865 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier AdaBoost
[80,20] [Train,Test] Split
Training for 50 classes.
Took 37.557

/home/max/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Took 28.8612320423 seconds
	Testing error is 0.0721442885772
Took 0.0796740055084 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier LinearSvm
[50,50] [Train,Test] Split
Training for 50 classes.
Took 0.460148096085 seconds
	Testing error is 0.0721442885772
Took 0.0801529884338 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier DecisionTree
[50,50] [Train,Test] Split
Training for 50 classes.
Took 0.194403171539 seconds
	Testing error is 0.583166332665
Took 0.00602102279663 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier AdaBoost
[50,50] [Train,Test] Split
Training for 50 classes.
Took 22.6015930176 seconds
	Testing error is 0.294589178357
Took 0.231884002686 seconds
----------------------------------------------------

--------------------------

/home/max/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


Took 8.98284316063 seconds
	Testing error is 0.109022556391
Took 0.0684790611267 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier LinearSvm
[20,80] [Train,Test] Split
Training for 50 classes.
Took 0.171267986298 seconds
	Testing error is 0.298245614035
Took 0.0671751499176 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier DecisionTree
[20,80] [Train,Test] Split
Training for 50 classes.
Took 0.0666217803955 seconds
	Testing error is 0.745614035088
Took 0.00832009315491 seconds
----------------------------------------------------

----------------------------------------------------
Using classifier AdaBoost
[20,80] [Train,Test] Split
Training for 50 classes.
Took 8.37769007683 seconds
	Testing error is 0.540100250627
Took 0.225660085678 seconds
----------------------------------------------------

---------------------------